# Visualización Atómica
Por Benjamín Astete

---
Este archivo contiene el proceso completo de carga, procesamiento y visualización de los datos de importaciones en Chile utilizando Altair para nuestra investigación.

In [17]:
# --- 1. Instalación de dependencias ---
!pip install altair vega_datasets pandas matplotlib cairosvg altair_saver vl-convert-python
# Install system dependencies for cairosvg
!apt-get install -y libcairo2 libpango1.0-0 libgdk-pixbuf2.0-0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.3/89.3 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.0/33.0 MB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 844.5/844.5 kB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 74.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.0/512.0 kB 31.2 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libcairo2 is already the newest version (1.16.0-5ubuntu2).
libgdk-pixbuf2.0-0 is already the newest version (2.40.2-2build4).
libpango1.0-0 is already the newest version (1.50.6+ds-2ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.


In [1]:
import pandas as pd

from google.colab import files
uploaded = files.upload()

# Assuming the uploaded file name is the key in the 'uploaded' dictionary
file_name = list(uploaded.keys())[0]

df = pd.read_csv(file_name, encoding='latin-1')
display(df.head())

Saving Importaciones.csv to Importaciones.csv


,AÃ±o,Mes,Vestuario,Calzado,Perfumes,Carne,Otros alimentos,Bebidas y alcoholes
0,2014,Enero,277,124,51,74,77,29
1,2014,Febrero,314,114,67,75,72,28
2,2014,Marzo,264,66,59,65,83,32
3,2014,Abril,165,43,65,82,84,31
4,2014,Mayo,193,60,58,76,87,32


In [4]:
# --- 3. Procesamiento de datos ---
# Convertir 'AÃ±o' y 'Mes' a formato de fecha mensual (por ejemplo: '2014-01-01')
month_mapping = {
    'Enero': 1, 'Febrero': 2, 'Marzo': 3, 'Abril': 4, 'Mayo': 5, 'Junio': 6,
    'Julio': 7, 'Agosto': 8, 'Septiembre': 9, 'Octubre': 10, 'Noviembre': 11, 'Diciembre': 12
}
df['Mes_Numero'] = df['Mes'].map(month_mapping)
df['Fecha'] = pd.to_datetime(df['AÃ±o'].astype(str) + '-' + df['Mes_Numero'].astype(str) + '-01')
df = df.sort_values('Fecha')

# Drop the temporary 'Mes_Numero' column
df = df.drop(columns=['Mes_Numero'])

# Identificar columnas a normalizar (ej. cantidades importadas)
columns_to_normalize = [
    'Vestuario', 'Calzado', 'Perfumes', 'Carne',
    'Otros alimentos', 'Bebidas y alcoholes'
]

# Limpieza y conversión de columnas a numérico
for col in columns_to_normalize:
    # Asegurar que la columna sea tipo string antes de reemplazar y luego convertir a numérico
    df[col] = df[col].astype(str).str.replace(',', '', regex=False)
    df[col] = pd.to_numeric(df[col], errors='coerce')
    # Rellenar valores NaN con 0 después de la conversión a numérico
    df[col] = df[col].fillna(0)

# Normalización de indicadores
for col in columns_to_normalize:
    min_val = df[col].min()
    max_val = df[col].max()
    if (max_val - min_val) != 0: # Evitar división por cero si todos los valores son iguales
        df[f'{col}_normalizado'] = (df[col] - min_val) / (max_val - min_val)
    else:
        df[f'{col}_normalizado'] = 0 # Si todos los valores son iguales, el valor normalizado es 0

display(df.head())

,AÃ±o,Mes,Vestuario,Calzado,Perfumes,Carne,Otros alimentos,Bebidas y alcoholes,Fecha,Vestuario_normalizado,Calzado_normalizado,Perfumes_normalizado,Carne_normalizado,Otros alimentos_normalizado,Bebidas y alcoholes_normalizado
0,2014,Enero,277,124,51,74,77,29,2014-01-01,0.490716,0.777778,0.085366,0.103896,0.129032,0.250000
1,2014,Febrero,314,114,67,75,72,28,2014-02-01,0.588859,0.698413,0.280488,0.110390,0.075269,0.230769
2,2014,Marzo,264,66,59,65,83,32,2014-03-01,0.456233,0.317460,0.182927,0.045455,0.193548,0.307692
3,2014,Abril,165,43,65,82,84,31,2014-04-01,0.193634,0.134921,0.256098,0.155844,0.204301,0.288462
4,2014,Mayo,193,60,58,76,87,32,2014-05-01,0.267905,0.269841,0.170732,0.116883,0.236559,0.307692


In [12]:
# --- 4. Visualización con Altair ---
import altair as alt

alt.data_transformers.disable_max_rows()

chart = alt.Chart(df).mark_line(point=True).encode(
    x=alt.X('Fecha:T', title='Fecha'),
    y=alt.Y('value:Q', title='Cantidad de Importación (Millones de dólares)'), # Actualizar el título del eje Y
    color=alt.Color('variable:N', title='Categoría de Importación',
                    scale=alt.Scale(
                        domain=[
                            'Perfumes',
                            'Vestuario',
                            'Calzado',
                            'Bebidas y alcoholes',
                            'Otros alimentos',
                            'Carne'
                        ],
                        range=[
                            'pink',      # Perfumes
                            'deeppink',  # Vestuario
                            'firebrick', # Calzado
                            'lightgray', # Bebidas y alcoholes
                            'gray',      # Otros alimentos
                            'dimgray'    # Carne
                        ]
                    )),
    tooltip=['Fecha:T', 'variable:N', 'value:Q']
).transform_fold(
    [
        'Vestuario',
        'Calzado',
        'Perfumes',
        'Carne',
        'Otros alimentos',
        'Bebidas y alcoholes'
    ],
    as_=['variable', 'value']
).properties(
    title='Evolución mensual de las importaciones en Chile (2014–2024)',
    width=800,
    height=400
)

chart.interactive()

alt.Chart(...)

In [20]:
# --- 5. Exportación de visualización ---
import altair as alt
import cairosvg

# Exportar la visualización en HTML (formato interactivo)
chart.save('visualizacion_benjamin.html')

# Exportar la visualización en PNG, especificando el motor de conversión
chart.save('visualizacion_benjamin.png', engine='vl-convert')

# Exportar primero como SVG (formato vectorial)
chart.save('visualizacion_benjamin.svg')

# Convertir el SVG a JPG usando cairosvg
cairosvg.svg2png(url='visualizacion_benjamin.svg', write_to='visualizacion_benjamin.jpg', dpi=300)

print("✅ Archivos exportados correctamente:")
print(" - visualizacion_benjamin.html (interactivo)")
print(" - visualizacion_benjamin.png (imagen estática)")
print(" - visualizacion_benjamin.jpg (imagen estática)")

✅ Archivos exportados correctamente:
 - visualizacion_benjamin.html (interactivo)
 - visualizacion_benjamin.png (imagen estática)
 - visualizacion_benjamin.jpg (imagen estática)


In [ ]:
!pip install vl-convert-python